*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will create a book recommendation algorithm using **K-Nearest Neighbors**.

You will use the [Book-Crossings dataset](http://www2.informatik.uni-freiburg.de/~cziegler/BX/). This dataset contains 1.1 million ratings (scale of 1-10) of 270,000 books by 90,000 users. 

After importing and cleaning the data, use `NearestNeighbors` from `sklearn.neighbors` to develop a model that shows books that are similar to a given book. The Nearest Neighbors algorithm measures distance to determine the “closeness” of instances.

Create a function named `get_recommends` that takes a book title (from the dataset) as an argument and returns a list of 5 similar books with their distances from the book argument.

This code:

`get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")`

should return:

```
[
  'The Queen of the Damned (Vampire Chronicles (Paperback))',
  [
    ['Catch 22', 0.793983519077301], 
    ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479], 
    ['Interview with the Vampire', 0.7345068454742432],
    ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
    ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]
  ]
]
```

Notice that the data returned from `get_recommends()` is a list. The first element in the list is the book title passed in to the function. The second element in the list is a list of five more lists. Each of the five lists contains a recommended book and the distance from the recommended book to the book passed in to the function.

If you graph the dataset (optional), you will notice that most books are not rated frequently. To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.

The first three cells import libraries you may need and the data to use. The final cell is for testing. Write all your code in between those cells.

In [3]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
from sklearn import preprocessing
import sklearn
import matplotlib.pyplot as plt

In [4]:
books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

In [5]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [6]:
# dropping books duplicates
df_books = df_books.drop_duplicates()

# only users with 200+ reviews
counts1 = df_ratings['user'].value_counts()
df_ratings = df_ratings[df_ratings['user'].isin(counts1[counts1 >= 200].index)]

In [7]:
# merging the two dataframes
df = pd.merge(df_books, df_ratings, on='isbn')

# dropping NaN values caused by the merge
df = df.dropna(axis=0, subset=['title'])

In [8]:
# counting the number of reviews per unique isbn value
ratingCount = (df.
               groupby(by=['isbn'])['rating'].
               count().
               reset_index().
               rename(columns = {'rating': 'ratingCount'})
               [['isbn', 'ratingCount']]
               )

In [9]:
# using the number of reviews as threshold for statistical significance
threshold = 30
df_ratingcount = df.merge(ratingCount, left_on='isbn', right_on='isbn', how='left')
df_ratingcount = df_ratingcount.query('ratingCount >= @threshold')

# dropping possible duplicates
df_ratingcount = df_ratingcount.drop_duplicates(['user', 'title'])

In [10]:
# creating a pivot table with unique users as columns
df_ratingcount_pivot = df_ratingcount.pivot_table(index='isbn', columns='user', values='rating').fillna(0)

In [11]:
# reseting index in order to be able to map isbn values to book title
df_ratingcount_pivot = df_ratingcount_pivot.reset_index()
df_ratingcount_pivot['isbn'] = df_ratingcount_pivot['isbn'].map(df_books.set_index('isbn')['title'])
df_ratingcount_pivot.index = df_ratingcount_pivot.isbn

# dropping useless column
df_ratingcount_pivot = df_ratingcount_pivot.drop(df_ratingcount_pivot.columns[[0]], axis=1)

In [12]:
# creating csr matrix to improve comutation
df_ratingcount_matrix = csr_matrix(df_ratingcount_pivot.values)

# creating the model
knn = NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(df_ratingcount_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [13]:
# function that returns the recommended books
def get_recommends(title):
  main = []
  k = 5
  e_dist, indices = knn.kneighbors(df_ratingcount_pivot.iloc[df_ratingcount_pivot.index == title].values.reshape(1,-1), n_neighbors=k+1)
  for i in range(0, 2):
    if i == 0:
      continue
    else:
      main.append([[df_ratingcount_pivot.index[indices.flatten()[i]], e_dist.flatten()[i]] for i in range(1,k+1)])
  main[0].reverse()
  main.insert(0, title)
  return main

In [14]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['Blue Diary', 0.7688525], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
You passed the challenge! 🎉🎉🎉🎉🎉


Use the cell below to test your function. The `test_book_recommendation()` function will inform you if you passed the challenge or need to keep trying.